In [1]:
import numpy as np
import scipy.io as sio
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

In [16]:
def splitMatrix(matrix):
    diagonalMatrix = np.diag(np.log(np.diag(matrix)))
    strictlyLowerTriangularMatrix = np.tril(matrix, k = -1)
    return diagonalMatrix, strictlyLowerTriangularMatrix

In [17]:
def distanceMatrices(matrix1, matrix2):
    chol1 = np.linalg.cholesky(matrix1)
    chol2 = np.linalg.cholesky(matrix2)
    chol1D, chol1L = splitMatrix(chol1)
    chol2D, chol2L = splitMatrix(chol2)
    distanceL = np.square(np.linalg.norm(chol1L - chol2L, 'fro'))
    distanceD = np.square(np.linalg.norm(chol1D - chol2D, 'fro'))
    distance = np.sqrt(distanceL + distanceD)
    return distance

In [18]:
def pairwiseDistances(matrix):
    numberMatrices, n, _ = np.shape(matrix)
    DISTANCES = np.zeros((numberMatrices, numberMatrices))
    for i in range(numberMatrices):
        for j in range(i, numberMatrices):
            distance = distanceMatrices(matrix[i], matrix[j])
            DISTANCES[i, j] = distance
            DISTANCES[j, i] = distance
    return DISTANCES

In [50]:
def tSNE(covarianceMatrix, Perplexity):
    temporaryMatrix = covarianceMatrix.reshape(-1, 12, 12)
    
    pairwiseDistance = pairwiseDistances(temporaryMatrix)
    
    tsne = TSNE(metric = 'precomputed', perplexity = Perplexity, learning_rate = 200, early_exaggeration = 4, init = "random", n_jobs = -1)

    out = tsne.fit_transform(pairwiseDistance)


    return out

In [51]:
covarianceMatrices = np.load("UCDMyoVerseSubject0Data.npy")
Labels = np.load("UCDMyoVerseSubject0Labels.npy")
print(covarianceMatrices.shape)
print(Labels.shape)

tsneEmbedding = tSNE(covarianceMatrices, 20)

colors = colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#00FFFF', '#FF00FF', '#FFA500', '#800080', '#A52A2A', '#FFC0CB']

colors = ListedColormap(colors)

plt.scatter(tsneEmbedding[:, 0], tsneEmbedding[:, 1], c = Labels, cmap = colors, marker='.')

colorbar_ax = plt.gcf().add_axes([0.92, 0.1, 0.02, 0.8])
                                
cbar = plt.colorbar(orientation='vertical', cax=colorbar_ax)
cbar.set_ticks(range(10))  
cbar.set_ticklabels(range(10)) 
plt.show()

(360, 12, 12)
(360,)


In [54]:
covarianceMatrices = np.load("ninaproSubject0Data.npy")
Labels = np.load("ninaproSubject0Labels.npy")
print(covarianceMatrices.shape)
print(Labels.shape)

tsneEmbedding = tSNE(covarianceMatrices, 5)

colors = [

"darkred",
"darkblue",
"darkgreen",
"darkcyan",
"darkmagenta",
"darkorange",
"darkolivegreen",
"darkorchid",
"darkgoldenrod",
"darkseagreen",
"darkslateblue",
"darkslategray",
"darkturquoise",
"darkviolet",
"darkkhaki",
"indigo",
"firebrick",]

colors = ListedColormap(colors)

plt.scatter(tsneEmbedding[:, 0], tsneEmbedding[:, 1], c = Labels, cmap = colors, marker='.')

colorbar_ax = plt.gcf().add_axes([0.92, 0.1, 0.02, 0.8])
                                 
cbar = plt.colorbar(orientation='vertical', cax=colorbar_ax)
cbar.set_ticks(range(17))  
cbar.set_ticklabels(range(17)) 
plt.show()
plt.close()



(102, 12, 12)
(102,)
